In [ ]:
##### importing required packages
import numpy as np
import collections
from math import log
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
#tree node storing required information
class treeNode:
    def __init__(self):
        self.yes = None   #left child
        self.no = None    #right child
        self.feature = None    #feature on the basis of which this node will be split into 2 in the next step
        self.entropy = None    #entopy of current node
        self.total_samples = None   #Total number of elementsit contains
        self.values = None          #Values of different classes stored in the form of dictionary
        self.ans = None             #storing majority answer of this node
        self.breakPoint = None      #storing the condition on the basis of which the node will be split

In [3]:
#returns split info to help calculate information gain
#this functions requires only 3 arguments because we are making only a binary tree
def splitInfo(x_train,x1,x2):
    a = len(x1)
    b = len(x2)
    t = len(x_train)
    ans = (-(a/t)*log(a/t , 2)) - ((b/t)*log(b/t,2))
    return ans

In [4]:
#function to return entropy
#count_of_classes is a dictionary which stores the composition of output array
# key = class, value = number of elements of that class
def getEntropy(count_of_classes):
    total = 0
    for key in count_of_classes:
        total += count_of_classes[key]
    ans = 0
    for key in count_of_classes:
            temp = count_of_classes[key] / total
            if(temp==0):
                continue
            ans = ans - (temp * log(temp,2))
    return ans

In [5]:
#function to return gain ratio
def getGainRatio(x_train,x1,x2,classes):
    if(len(x1)==0 or len(x2)==0):
        return -1
    #count respective instances of classes ans store them in the form of dictionary    
    x_train_classes = collections.Counter(x_train[:,len(x_train[0])-1])
    x1_classes = collections.Counter(x1[:,len(x1[0])-1])
    x2_classes = collections.Counter(x2[:,len(x2[0])-1])
    
    #calculate information gain
    info_gain = getEntropy(x_train_classes) - ((len(x1)/len(x_train))*getEntropy(x1_classes) + (len(x2)/len(x_train))*getEntropy(x2_classes))
    
    #calculate gain ratio and return 
    gain_ratio = info_gain/splitInfo(x_train,x1,x2)
    return gain_ratio

In [6]:
#function to return the index of the feature on which node has to be split
def getSplitFeature(x_train,features,classes):
    max_feature = 0
    max_gain= -1
    breakPoint = 0
    # check for all features
    for f in features:
        arr = x_train[:,f]
        #sorting the array to calculate mid points 
        np.sort(arr)
        mid_list = []      #list storing mid points
        for i in range(len(arr)-1):
            temp = (arr[i] + arr[i+1])/2
            mid_list.append(temp)
        #checking for each point
        for i in range(len(mid_list)):
            condition = x_train[:,f] < mid_list[i]
            x1 = x_train[condition==True]
            x2 = x_train[condition==False]
            temp = getGainRatio(x_train,x1,x2,classes)
            #calculating max and updating values accordingly
            if(temp>max_gain):
                max_gain = temp
                breakPoint = mid_list[i]
                max_feature = f
    return breakPoint,max_feature,max_gain

In [7]:
#function to build tree
def buildTree(x_train,features,classes,level):
    
    count_of_classes = collections.Counter(x_train[:,len(x_train[0])-1])    #dictionary storing count of classes
    #filling respective values of root node 
    root = treeNode()
    root.entropy = getEntropy(count_of_classes)
    root.values = count_of_classes
    root.total_samples = len(x_train)
    max_here = -1
    max_key = None
    for key in count_of_classes:
        if(count_of_classes[key]>max_here):
            max_here = count_of_classes[key]
            max_key = key 
    root.ans = max_key
    #printing required output
    print("Level",level)
    for i in classes:
        if(count_of_classes[i]!=0):
            print("Count of ",i," = ",count_of_classes[i])
    print("Current Entropy is = ",root.entropy)
    
    #base case 1
    #if features have been exhausted
    if (len(features)==0):
        print("Reached Leaf Node")
        print()
        return None
    
    #base case 2
    #if node is pure node
    if(len(count_of_classes) == 1):
        #pure node found
        #no need to make further recursive calls
        print("Reached Leaf Node")
        print()
        return root
    
    #get the feature to split on
    break_point, feature_used,gain = getSplitFeature(x_train,features,classes)
    print("Splitting on feature",feature_used,"with gain ratio = ",gain)
    
    #divide the training data into 2 parts to be split as left and right now
    index = features.index(feature_used)
    features.remove(feature_used)
    expr = x_train[:,feature_used] < break_point
    new_x_yes = x_train[expr == True]
    new_x_no = x_train[expr == False]
    print()
    
    root.feature = feature_used
    root.breakPoint = break_point
    
    
    #recursive call on left and right subtrees 
    root.yes = buildTree(new_x_yes,features,classes,level+1)
    root.no = buildTree(new_x_no,features,classes,level+1)
    
    #insert the feature index back again
    features.insert(index,feature_used)
    
    return root
    
    
    

In [8]:
#train data
def fit(data):
    x_train = data[:,:] 
    features = [i for i in range(len(x_train[0])-1)]
    classes = collections.Counter(x_train[:,len(x_train[0])-1])
    
    tree = buildTree(x_train,features,classes,0)
    return tree

In [9]:
#recursive function to return answer for one instance at a time
def getAns(x,root):
    #reached leaf
    if(root.yes == None and root.no == None):
        return root.ans
    else:
        f = root.feature
        bp = root.breakPoint
        #check which node to go to next
        if(x[f]<bp):
            return getAns(x,root.yes)
        else:
            return getAns(x,root.no)

In [10]:
#return predictions
def predict(x,root):
    y=np.zeros(len(x))
    for i in range(len(x)):
        y[i] = getAns(x[i],root)
    return y

In [11]:
#main function to load data and run fit function
def main():
    #example 1
    #Or dataset as given in the problem
    data = np.array([[1,1,1],
                   [1,0,1],
                   [0,1,1],
                   [0,0,0]])
    print("Output of example 1")
    root = fit(data)
    print()
    print()
    print()
    
    #example 2 
    # Iris datset from sklearn    
    iris = datasets.load_iris()
    iris.target = iris.target.reshape(-1,1)
    arr = np.concatenate((iris.data,iris.target),axis=1)
    print("Output of example 2")
    root = fit(arr)
        
    #root has now been obtained
    # Here the training data is used for testing as well just to test the working of the tree
    #The tree predicts correctly for most of the cases
    x_test = iris.data
    y_pred = predict(x_test,root)
    np.savetxt("out.csv",y_pred,delimiter=",")

In [12]:
#run main
main()

Output of example 1
Level 0
Count of  1  =  3
Count of  0  =  1
Current Entropy is =  0.8112781244591328
Splitting on feature 0 with gain ratio =  0.31127812445913283

Level 1
Count of  1  =  1
Count of  0  =  1
Current Entropy is =  1.0
Splitting on feature 1 with gain ratio =  1.0

Level 2
Count of  0  =  1
Current Entropy is =  0.0
Reached Leaf Node

Level 2
Count of  1  =  1
Current Entropy is =  0.0
Reached Leaf Node

Level 1
Count of  1  =  2
Current Entropy is =  0.0
Reached Leaf Node




Output of example 2
Level 0
Count of  0.0  =  50
Count of  1.0  =  50
Count of  2.0  =  50
Current Entropy is =  1.584962500721156
Splitting on feature 3 with gain ratio =  0.9999999999999999

Level 1
Count of  0.0  =  50
Current Entropy is =  0.0
Reached Leaf Node

Level 1
Count of  1.0  =  50
Count of  2.0  =  50
Current Entropy is =  1.0
Splitting on feature 2 with gain ratio =  0.6621582046482519

Level 2
Count of  1.0  =  44
Count of  2.0  =  1
Current Entropy is =  0.1537421803287619
Spli